<a href="https://colab.research.google.com/github/RuslanMavlitov/IDE/blob/master/TSYM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задача проанализировать суточное потребление по восьми трансформаторам по логам с счетчиков на низкой стороне трансформаторов.

In [2]:
# Загрузим библиотеки для работы с данными
import pandas as pd
import numpy as np
# Будем использовать Plotly для визуализации
import plotly.express as px
import plotly.graph_objects as go
# Все данные будем брать с Гугл-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Анализ данных

### 1.1 Чтение данных

In [3]:
#Загрузим данные
T1 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T1_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T2 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T2_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T3 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T3_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T4 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T4_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T5 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T5_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T6 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T6_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T7 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T7_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T8 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T8_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')

In [126]:
#Посмотрим что получили
T1.head()

,Äàòà,Âðåìÿ,Òî÷êà,Çíà÷åíèå,Òèï
0,03.07.2024,16:31:05.000,SHA_GRSCH_RP4_PK47_AI210,<634>,NaN
1,03.07.2024,16:31:05.000,SHA_GRSCH_RP4_PK47_AI212,<651>,NaN
2,03.07.2024,16:32:25.000,SHA_GRSCH_RP4_PK47_AI210,<647>,NaN
3,03.07.2024,16:32:33.000,SHA_GRSCH_RP4_PK47_AI210,<617>,NaN
4,03.07.2024,16:32:39.000,SHA_GRSCH_RP4_PK47_AI212,<638>,NaN


Данные нужно преобразовать:
- переименуем столбцы
- дату и время в один формат по времени
- токи в счетчике переименуем в номера фаз
- удалим лишние знаки в значении тока


### 1.2 Обработка данных

In [4]:
#Удалим последний столбец без данных
T1.drop(T1.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T1.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T1['Current'] = T1['Current'].str.replace('<', '', regex=True)
T1['Current'] = T1['Current'].str.replace('>', '', regex=True)
T1['Current'] = T1['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T1['datetime'] = pd.to_datetime(T1['data'] + " " + T1['Time'])
T1['datetime'] = T1['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T1 = T1.set_index(pd.DatetimeIndex(T1['datetime']))
T1 = T1.drop(['datetime'], axis = 1)
T1.drop(['data','Time'], axis=1, inplace=True)

In [142]:
#Посмотрим сколько уникальных имен датчиков тока в данных
print(f"В данных всего {T1['Multimetr_name'].nunique()} уникальных датчика тока")

В данных всего 2 уникальных датчика тока


In [5]:
T1['Multimetr_name'] = T1['Multimetr_name'].replace({'SHA_GRSCH_RP4_PK47_AI210' : 'Phase_1', 'SHA_GRSCH_RP4_PK47_AI212' : 'Phase_2'})

In [137]:
#Посмотрим на результат
T1.head(2)

,Multimetr_name,Current
datetime,,
2024-07-03 16:31:05,Phase_1,634.0
2024-07-03 16:31:05,Phase_2,651.0


Уже намного лучше... Проделаем эту обработку по всем счетчикам

In [6]:
#Удалим последний столбец без данных
T2.drop(T2.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T2.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T2['Current'] = T2['Current'].str.replace('<', '', regex=True)
T2['Current'] = T2['Current'].str.replace('>', '', regex=True)
T2['Current'] = T2['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T2['datetime'] = pd.to_datetime(T2['data'] + " " + T2['Time'])
T2['datetime'] = T2['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T2 = T2.set_index(pd.DatetimeIndex(T2['datetime']))
T2 = T2.drop(['datetime'], axis = 1)
T2.drop(['data','Time'], axis=1, inplace=True)

T2['Multimetr_name'] = T2['Multimetr_name'].replace(
    {'SHA_GRSCH_RP3_PK47_AI223' : 'Phase_1',
     'SHA_GRSCH_RP3_PK47_AI224' : 'Phase_2',
     'SHA_GRSCH_RP3_PK47_AI225' : 'Phase_3'})

In [7]:
#Удалим последний столбец без данных
T3.drop(T3.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T3.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T3['Current'] = T3['Current'].str.replace('<', '', regex=True)
T3['Current'] = T3['Current'].str.replace('>', '', regex=True)
T3['Current'] = T3['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T3['datetime'] = pd.to_datetime(T3['data'] + " " + T3['Time'])
T3['datetime'] = T3['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T3 = T3.set_index(pd.DatetimeIndex(T3['datetime']))
T3 = T3.drop(['datetime'], axis = 1)
T3.drop(['data','Time'], axis=1, inplace=True)

T3['Multimetr_name'] = T3['Multimetr_name'].replace(
    {'SHA_GRSCH_RP2_PK47_AI236' : 'Phase_1',
     'SHA_GRSCH_RP2_PK47_AI237' : 'Phase_2',
     'SHA_GRSCH_RP2_PK47_AI238' : 'Phase_3'})

In [8]:
#Удалим последний столбец без данных
T4.drop(T4.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T4.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T4['Current'] = T4['Current'].str.replace('<', '', regex=True)
T4['Current'] = T4['Current'].str.replace('>', '', regex=True)
T4['Current'] = T4['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T4['datetime'] = pd.to_datetime(T4['data'] + " " + T4['Time'])
T4['datetime'] = T4['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T4 = T4.set_index(pd.DatetimeIndex(T4['datetime']))
T4 = T4.drop(['datetime'], axis = 1)
T4.drop(['data','Time'], axis=1, inplace=True)

T4['Multimetr_name'] = T4['Multimetr_name'].replace(
    {'SHA_GRSCH_RP1_PK47_AI249' : 'Phase_1',
     'SHA_GRSCH_RP1_PK47_AI250' : 'Phase_2',
     'SHA_GRSCH_RP1_PK47_AI251' : 'Phase_3'})

In [9]:
#Удалим последний столбец без данных
T5.drop(T5.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T5.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T5['Current'] = T5['Current'].str.replace('<', '', regex=True)
T5['Current'] = T5['Current'].str.replace('>', '', regex=True)
T5['Current'] = T5['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T5['datetime'] = pd.to_datetime(T5['data'] + " " + T5['Time'])
T5['datetime'] = T5['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T5 = T5.set_index(pd.DatetimeIndex(T5['datetime']))
T5 = T5.drop(['datetime'], axis = 1)
T5.drop(['data','Time'], axis=1, inplace=True)

T5['Multimetr_name'] = T5['Multimetr_name'].replace(
    {'GRSCH_PM31_A1' : 'Phase_1',
     'GRSCH_PM31_A2' : 'Phase_2',
     'GRSCH_PM31_A3' : 'Phase_3'})

In [10]:
#Удалим последний столбец без данных
T6.drop(T6.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T6.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T6['Current'] = T6['Current'].str.replace('<', '', regex=True)
T6['Current'] = T6['Current'].str.replace('>', '', regex=True)
T6['Current'] = T6['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T6['datetime'] = pd.to_datetime(T6['data'] + " " + T6['Time'])
T6['datetime'] = T6['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T6 = T6.set_index(pd.DatetimeIndex(T6['datetime']))
T6 = T6.drop(['datetime'], axis = 1)
T6.drop(['data','Time'], axis=1, inplace=True)

T6['Multimetr_name'] = T6['Multimetr_name'].replace(
    {'GRSCH_PM32_A1' : 'Phase_1',
     'GRSCH_PM32_A2' : 'Phase_2',
     'GRSCH_PM32_A3' : 'Phase_3'})

In [11]:
#Удалим последний столбец без данных
T7.drop(T7.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T7.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T7['Current'] = T7['Current'].str.replace('<', '', regex=True)
T7['Current'] = T7['Current'].str.replace('>', '', regex=True)
T7['Current'] = T7['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T7['datetime'] = pd.to_datetime(T7['data'] + " " + T7['Time'])
T7['datetime'] = T7['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T7 = T7.set_index(pd.DatetimeIndex(T7['datetime']))
T7 = T7.drop(['datetime'], axis = 1)
T7.drop(['data','Time'], axis=1, inplace=True)

T7['Multimetr_name'] = T7['Multimetr_name'].replace(
    {'GRSCH_PM41_A1' : 'Phase_1',
     'GRSCH_PM41_A2' : 'Phase_2',
     'GRSCH_PM41_A3' : 'Phase_3'})

In [12]:
#Удалим последний столбец без данных
T8.drop(T8.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T8.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T8['Current'] = T8['Current'].str.replace('<', '', regex=True)
T8['Current'] = T8['Current'].str.replace('>', '', regex=True)
T8['Current'] = T8['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T8['datetime'] = pd.to_datetime(T8['data'] + " " + T8['Time'])
T8['datetime'] = T8['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T8 = T8.set_index(pd.DatetimeIndex(T8['datetime']))
T8 = T8.drop(['datetime'], axis = 1)
T8.drop(['data','Time'], axis=1, inplace=True)

T8['Multimetr_name'] = T8['Multimetr_name'].replace(
    {'GRSCH_PM42_A1' : 'Phase_1',
     'GRSCH_PM42_A2' : 'Phase_2',
     'GRSCH_PM42_A3' : 'Phase_3'})

### Визуализация данных

In [13]:
# Построим график нагрузки Т-1
fig = px.line(T1, x=T1.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-1",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=1400, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=1400/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [14]:
# Построим график нагрузки Т-2
fig = px.line(T2, x=T2.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-2",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=1400, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=1400/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [15]:
# Построим график нагрузки Т-3
fig = px.line(T3, x=T3.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-3",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=1400, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=1400/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [16]:
# Построим график нагрузки Т-4
fig = px.line(T4, x=T4.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-4",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=1400, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=1400/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [17]:
# Построим график нагрузки Т-5
fig = px.line(T5, x=T5.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-5",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=2300, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=2300/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [18]:
# Построим график нагрузки Т-6
fig = px.line(T6, x=T6.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-6",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=2300, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=2300/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [19]:
# Построим график нагрузки Т-7
fig = px.line(T7, x=T7.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-7",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=2300, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=2300/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()

In [20]:
# Построим график нагрузки Т-8
fig = px.line(T8, x=T8.index, y="Current", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-8",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Ток, А"),
    )
fig.add_hline(y=2300, line_dash="dot",
              annotation_text="100% от Iном",
              annotation_position="bottom right")
fig.add_hline(y=2300/2,  line_dash="dot",
              annotation_text="50% от Iном",
              annotation_position="bottom right")

fig.show()